In [18]:
import json
import subprocess
from pathlib import Path
import geopandas as gpd
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
from pdal import Reader, Filter, Writer, Pipeline

In [19]:
def run_pdal(pipeline: Pipeline, pipe_name="pdal", args = []):
    # don't use the PDAL pythonb bindings to run the pipeline, but make an external command call
    # this is because the PDAL python bindings are not thread safe
    # and they have the tendency to crash the python interpreter, while if they are a separate process there is no problem
    # for this use case we don't need to pass data between the python code and the PDAL pipeline so it pefectly fine to use the command line interface
    with open(f"pipeline_{pipe_name}.json", "w") as f:
        json.dump(pipeline.toJSON(), f)
    cmd = ["pdal", "pipeline", f"pipeline_{pipe_name}.json"]
    cmd.extend(args)
    subprocess.run(cmd, check=True)

# Processing steps

In [20]:
base_dir = Path("/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant")

In [21]:
raw_dir = base_dir / "0_raw_lidar"
denoise_dir = base_dir / "1_denoised_lidar"
ground_dir = base_dir / "2_ground_lidar"
dtm_dir = base_dir / "3_dtm_raster"
norm_dir = base_dir / "4_norm_lidar"
chm_dir = base_dir / "5_chm_raster"

# create directories
raw_dir.mkdir(exist_ok=True)
denoise_dir.mkdir(exist_ok=True)
ground_dir.mkdir(exist_ok=True)
dtm_dir.mkdir(exist_ok=True)
norm_dir.mkdir(exist_ok=True)
chm_dir.mkdir(exist_ok=True)

## 0) Create raw tindex

In [22]:
file_spec = str(raw_dir / "*.laz")

In [23]:
!pdal tindex create -f GPKG raw_tindex.gpkg --filespec "{file_spec}"

^C


In [24]:
raw_tindex = gpd.read_file("raw_tindex.gpkg")

## 1) Denoise 

In [25]:
cloud_raw = Path(raw_tindex.iloc[0].location)

In [26]:
cloud_denoise = denoise_dir / f"denoise_{cloud_raw.stem}.copc.laz"

In [27]:
denoise_pipe = Pipeline([
    Reader.las(cloud_raw),
    Filter.outlier(method="radius", radius=1.5, min_k=20), # we expect density of 5 points per square meter, 
    Filter.expression(expression="Classification!=7"),
    Writer.copc(cloud_denoise)
])

In [29]:
denoise_pipe.toJSON()

'[{"type": "readers.las", "filename": "/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/lidar_raw_0/PNOA_2023_CANT_376-4775_NPC02.laz", "tag": "readers_las1"}, {"type": "filters.outlier", "method": "radius", "radius": 1.5, "min_k": 20, "tag": "filters_outlier1"}, {"type": "filters.expression", "expression": "Classification!=7", "tag": "filters_expression1"}, {"type": "writers.copc", "filename": "/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/1_denoised_lidar/denoise_PNOA_2023_CANT_376-4775_NPC02.copc.laz", "tag": "writers_copc1"}]'

In [28]:
run_pdal(denoise_pipe, "denoise")

PDAL: Pipeline: root element is not a pipeline.



CalledProcessError: Command '['pdal', 'pipeline', 'pipeline_denoise.json']' returned non-zero exit status 1.